# Description VQA

In [ ]:
 '''
 OUR MODEL
Input: VQA2, Train: mscoco2014 train, Validation mscoco2014 val ,Test mscoco2015 test
https://visualqa.org/download.html
Trainval images: 123287(82,783 + 40,504)Trianval questions: 658111
Test images: 81434 Test questions:  447793 Test-dev images: 36807
Unique answers is 3133

Paper: 204,721 images from Microsoft COCO dataset,(443,757 train, 214,354 val, and 447,793 test questions)
DIFFERENCE :n_ans = 3133, only answers which appear more than 8 times in val+train dataset
Github : https://github.com/cvlab-tohoku/Dense-CoAttention-Network/blob/master/dense_coattn/cost/costs.py
 '''
    
'''
this model use PNASNet for visual feature selection and use dense layer to select the 
'''
    

In [1]:
import tensorflow as tf
import os
import sys
import cv2
import numpy as np
import tensorflow.contrib.slim as slim
import tensorflow_hub as hub
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(gpu_options=gpu_options,log_device_placement=True, allow_soft_placement=True)
import lmdb
%matplotlib inline
import matplotlib.pyplot as plt
import random
import pickle
#from bert_embedding import BertEmbedding

os.environ["TFHUB_CACHE_DIR"] = "/home/david_hc95/tfhub_modules"
# Jupyter caches the utils module, so changes there aren't reflected in a re-import. We delete the utils module from
# sys.modules first so that we force-reload all the code in utils.py

sys.path.append('../utilities/')
if 'utils_data_gen' in sys.modules:
    del sys.modules['utils_data_gen']
from utils_data_gen import *

if 'utils_evaluation' in sys.modules:
    del sys.modules['utils_evaluation']
from utils_evaluation import *

if 'utils_model' in sys.modules:
    del sys.modules['utils_model']
from utils_model import *

print('Import done')


W0513 00:16:16.581567 139913678259584 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Import done


In [ ]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

# Data Path

In [ ]:
if cloud ==1:
    os.environ["TFHUB_CACHE_DIR"] = "/home/sonamgoenka/tfhub_modules"
    slim_models_path = '/home/david_hc95/grounding/iccv19_grounding/models/'
    sys.path.append(slim_models_path)
    data_path = '/home/ha2436/data/'
else:
    os.environ["TFHUB_CACHE_DIR"] = "/home/sonam/tfhub_modules"
    slim_models_path = '/home/sonam/models/'
    sys.path.append(slim_models_path)
    data_path = '/dvmm-filer2/datasets/Groundings/data/

# Loading Data

In [2]:
base_path = '/home/ha2436/data/'
data_path = base_path + 'mscoco/'
lmdb_path = data_path + 'MSCOCO_jpg.lmdb'
lmdb_path_new = '/home/david_hc95/mscoco_test2015' 
vqa_base_path = '/home/david_hc95/VQA_pickles_new/'

dict_paths = {'train':'trainval.pickle', 'val':'val.pickle', 'test': 'test.pickle'} 
ans_paths = 'top_answer_list.pickle'

In [3]:
#loading MSCOCO img data 
lmdb_env = lmdb.open(lmdb_path, map_size=int(1e11), readonly=True, lock=False)
txn = lmdb_env.begin(write=False)
lmdb_env_test = lmdb.open(lmdb_path_new, map_size=int(1e11), readonly=True, lock=False)
txn_test = lmdb_env_test.begin(write=False)

with open(vqa_base_path + dict_paths['train'], 'rb') as f:
    dict_train = pickle.load(f, encoding='latin1')
    ids_train = list(dict_train.keys())

with open(vqa_base_path + "idx2ans.pickle", 'rb') as file:
    idx2ans = pickle.load(file)
    
with open(vqa_base_path + dict_paths['test'], 'rb') as f:
    dict_test = pickle.load(f, encoding='latin1')
    ids_test = list(dict_test.keys())

print('training image size', len(ids_train))
print('testing image size', len(ids_test))

training image size 123287
testing image size 81434


In [4]:
# Storing images in memory
def preprocess_images_vqa(all_ids_list, db, db2):
    valid_ids = []
    images_dict = {}
    for i, id_train in enumerate(all_ids_list):
        imgbin = db.get(id_train.encode('utf-8'))
        if imgbin == None:
            imgbin = db2.get(id_train.encode('utf-8'))
            if imgbin == None:
                raise Exception("image not found! %s" % id_train)
        buff = np.frombuffer(imgbin, dtype='uint8')        
        imgbgr = cv2.imdecode(buff, cv2.IMREAD_COLOR)
        img = imgbgr[:,:,[2,1,0]]
        im = cv2.resize(img,(299,299))  
        valid_ids.append(id_train)
        images_dict[id_train] = im
        if i % 10000 == 0:
            print('Processed: ', i, ' images')
    print('Done pre-processing images')            
    return valid_ids, images_dict

# Params

In [5]:
n_batch = 32
gamma_1 = 5.0
gamma_2 = 10.0
n_iter_per_epoch = int(len(dict_train)/n_batch)
# n_iter_per_epoch_val = 2  #int(5000/n_batch)
n_epochs = 15 #N of epochs
reg_val = .0005
n_ans = 3133 # number of answers

# visual params
lr_value = .001
log_file = './logs/vgg/vgg'

# Architecture
#Temp :'Phrase_Grounding_NewAttn_VG_Seg_COCO_Pyramid_VGG'
model_name =  './my_model/Model_' + 'VQA_LSTM_LearnableFuse_VQA2_VGG'
conv_kernel_size = 3 #1
n_layers = 1 #3
vis_method = depth_selection_vgg_pyramid # Pyramid
# segment_branch = 0
# msk_size = 37 #38 for pnasnet, 37 for vgg
visual_model = 'vgg' #pnasnet, vgg
text_model = "elmo" # bert
use_new_attn = False
use_pyramid = True
# latent_space_dim = 512

# MAX_SEQ_LENGTH = 50
# BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

# Attention

In [6]:
# this method feeds four attendance features to dense layer to generate a new attendance feature 
def fusion_learn(v, e_s):
    '''
    this function use cos similarity for attetion
    pass attended visual fetaure and sen emebedding to dense layer for level selection
    dense for feature fusion
    
    '''
    with tf.variable_scope('text_image_fusion'):
            ###sentence-level###
            #heatmap pool
            h_s = tf.nn.relu(tf.einsum('bj,blkj->blk',e_s,v)) #pair-wise e_bar*v^T: ?xNx4
            #attention
            a_s = tf.einsum('bjk,bjki->bik',h_s,v) #?xDx4 attnded visual reps for sen.
            #pair-wise score
            a_s_norm = tf.nn.l2_normalize(a_s,axis=1) # ? *D*4
            e_s_norm = tf.nn.l2_normalize(e_s,axis=1) # ?*D
            
            # take four attended visual feature and learn one representation
            visual = tf.contrib.layers.flatten(a_s_norm)   #?*4D
            visual = tf.layers.dense(visual, units=1024)
            visual = tf.nn.leaky_relu(visual,alpha=.25)
            visual = tf.layers.dense(visual, units=e_s_norm.get_shape().as_list()[1])
            
            fusion = tf.concat([e_s_norm, visual], axis=1) # ?*2D
            
            fusion = tf.layers.dense(fusion, units=1024)
            fusion = tf.nn.leaky_relu(fusion,alpha=.25)
            fusion = tf.layers.dense(fusion, units=1024)
            fusion = tf.nn.leaky_relu(fusion,alpha=.25)
            fusion = tf.layers.dense(fusion, units=1024)
            fusion = tf.nn.leaky_relu(fusion,alpha=.25)
            fusion = tf.layers.dense(fusion, units=1024)
            fusion = tf.nn.leaky_relu(fusion,alpha=.25)
            fusion = tf.layers.dense(fusion, units=n_ans)
            fusion = tf.identity(fusion, name='fusion')
            
            return fusion
# this method feeds four attendance features to dense layer to generate a new attendance feature 
def fusion_learn_pyramid(v, e_s):
    '''
    this function use cos similarity for attetion
    no level selection since we use pyramid
    dense for feature fusion
    
    '''
    with tf.variable_scope('text_image_fusion'):
            ###sentence-level###
            #heatmap pool
            h_s = tf.nn.relu(tf.einsum('bj,blkj->blk',e_s,v)) #pair-wise e_bar*v^T: ?xNx1
            #attention
            a_s = tf.einsum('bjk,bjki->bik',h_s,v) #?xDx1 attnded visual reps for sen.
            #pair-wise score
            a_s_norm = tf.nn.l2_normalize(a_s,axis=1) # ?xDx1
            e_s_norm = tf.nn.l2_normalize(e_s,axis=1) # ?xD
            
            # take four attended visual feature and learn one representation
            visual = tf.squeeze(a_s_norm, axis=-1) #?xD
            
            fusion = tf.concat([e_s_norm, visual], axis=-1) # ?x2D
            
            fusion = tf.layers.dense(fusion, units=1024)
            fusion = tf.nn.leaky_relu(fusion,alpha=.25)
            fusion = tf.layers.dense(fusion, units=1024)
            fusion = tf.nn.leaky_relu(fusion,alpha=.25)
            fusion = tf.layers.dense(fusion, units=1024)
            fusion = tf.nn.leaky_relu(fusion,alpha=.25)
            fusion = tf.layers.dense(fusion, units=1024)
            fusion = tf.nn.leaky_relu(fusion,alpha=.25)
            fusion = tf.layers.dense(fusion, units=n_ans)
            fusion = tf.identity(fusion, name='fusion')
            
            return fusion
        
        
def new_attn_fusion_learn(v, e_s):
    '''
    this function implement the attention and fusion of paper Tips and Tricks for Visual Question Answering:
    Learnings from the 2017 Challenge
    
    input:
        v: four level image feature ?xNx4x2048
        e_s: sentence feature ?x512
    return:
        attn: attendance feature of images #?x512
    '''
    # use dense layer to get one level of image feature
    def new_fusion(attn, e_s):
        '''
        this function fuse the attendance feature and sentence feature

        input:
            attn: attendance feature ?x512
            e_s: sentence feature ?xD

        return:
            fusion: final fusion ?x3133
        '''
        e_s = non_linear(e_s, 512)
        fusion = tf.math.multiply(attn, e_s)
        fusion = non_linear(fusion, 3133)
        fusion = tf.layers.dense(fusion, units=3133)
        return fusion
    
    
    def non_linear(in_tensor, num_units):
        '''
        This function implement the non-linear function in the paper 
        '''
        y_hat = tf.layers.dense(in_tensor, units=num_units)
        y_hat = tf.nn.tanh(y_hat)
        g = tf.layers.dense(in_tensor, units=num_units)
        g = tf.nn.sigmoid(g)
        y = tf.math.multiply(y_hat, g)

        return y
    
    # combine four level image features using dense layers
    v_shape = v.get_shape().as_list() 
    v = tf.reshape(v, [-1, v_shape[1], v_shape[2] * v_shape[3]])
    v = tf.layers.dense(v, units=2048)
    v = tf.nn.l2_normalize(v, axis=-1) # ?xNx2048
    
    
    # expand e_s to have the same compatible shape
    e_s_attn = tf.tile(tf.expand_dims(e_s, 1), [1,v.get_shape().as_list()[1], 1]) # ?xNxD check if this function is correct
    concat_sen_image = tf.concat([v, e_s_attn], axis=-1)
    
    attn = non_linear(concat_sen_image, 512) #?xNx512
    attn = tf.layers.dense(attn, units=1) #?xNx1
    attn = tf.squeeze(attn, -1) #?xN
    attn = tf.nn.softmax(attn, axis=-1) # ?xN   
    
    attn = tf.einsum('bj,bjk->bk',attn, v) # ?x512
    attn = non_linear(attn, 512) # ?x512
    
    fusion = new_fusion(attn, e_s)
    fusion = tf.identity(fusion, name='fusion')
    
    return fusion

    
def build_bilstm(w_embd,seq_length):
    with tf.variable_scope('BiLSTM'):
        # Forward direction cell
        lstm_fw_cell = tf.contrib.rnn.LSTMCell(512, forget_bias=1.0)
        # Backward direction cell
        lstm_bw_cell = tf.contrib.rnn.LSTMCell(512, forget_bias=1.0)
        outputs, _ = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, w_embd, sequence_length = seq_length,
                                                  dtype=tf.float32)
    output = tf.concat(outputs,axis=2,name='BiLSTM_out')
    return output



# Build Graph (Base)


In [7]:
tf.reset_default_graph()
try:
    sess.close()
except:
    print('Initialising for first time')
    
sess = tf.InteractiveSession(config=config)
mode = tf.placeholder(tf.string, name='mode')
isTraining = tf.equal(mode, 'train')
regularizer = tf.contrib.layers.l2_regularizer(reg_val)

with tf.device('/gpu:0'):
    
    print('Building Text Model...')        
    if text_model == "elmo":
        print("use elmo for textual model")
        #sentence placeholder - list of sentences
        text_batch = tf.placeholder('string', shape=[None], name='text_input')
        #loading pre-trained ELMo
        elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
        #getting ELMo embeddings
        elmo_embds = elmo(text_batch, signature="default", as_dict=True)
        w_embd = tf.identity(elmo_embds['word_emb'], name='elmo_word_embd') #?xTxD/2
        lstm_embd = build_bilstm(w_embd,elmo_embds['sequence_len']) #?xTxD
        #taking index of last word in each sentence
        idx = elmo_embds['sequence_len']-1
        batch_idx = tf.stack([tf.range(0,tf.size(idx),1),idx],axis=1)
        # Concatenate first of backward with last of forward to get sentence embeddings
        dim = lstm_embd.get_shape().as_list()[-1]
        sen_embd = tf.concat([lstm_embd[:,0,int(dim/2):],
        tf.gather_nd(lstm_embd[:,:,:int(dim/2)],batch_idx)], axis=-1) #[batch,dim]
        e_s = tf.layers.dense(sen_embd, units=1024)
        e_s = tf.nn.leaky_relu(e_s,alpha=.25)
        e_s = tf.layers.dense(e_s, units=1024)
        e_s = tf.nn.leaky_relu(e_s,alpha=.25)
        e_s = tf.nn.l2_normalize(e_s, axis=-1, name='sen_embedding')
    
    else:
        raise Exception("please type a valid text model")
    
    #Building visual model
    print('Building Visual Model...')
    input_img = tf.placeholder(tf.float32, (None,299,299,3), name='input_img')
    if visual_model == 'vgg':
        pre_processed_img = pre_process(input_img, 'vgg_preprocessing')
        vis_model = pre_trained_load(model_name='vgg_16', image_shape=(None,299,299,3),
                                  input_tensor=pre_processed_img, session=sess, is_training=False, global_pool=False)
    if visual_model == 'pnasnet':
        pre_processed_img = pre_process(input_img, 'inception_preprocessing')
        vis_model = pre_trained_load(model_name='pnasnet_large', image_shape=(None,299,299,3),
                                     input_tensor=pre_processed_img, session=sess, is_training=False, global_pool=True)
    v = vis_method(vis_model,regularizer, conv_kernel_size, n_layers) #(?,1225,4,1024)
    
    if not use_new_attn:
        print("use cosine simliarity attn mechanism")
        if use_pyramid:
            print("use pyramid")
            fusion = fusion_learn_pyramid(v, e_s)
        else:
            print("use level selection")
            fusion = fusion_learn(v, e_s)
    else:
        print("use new attn mechanism")
        fusion = new_attn_fusion_learn(v, e_s)
    
    answer_batch = tf.placeholder(tf.float32, shape=[n_batch, n_ans], name='answer_input')
    loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=fusion, labels=answer_batch)
    loss = tf.reduce_mean(loss)
    loss = tf.identity(loss, name='loss')

    tf.summary.scalar('loss_value', loss)

    lr = tf.placeholder(tf.float32, shape=[], name='learning_rate')
    tf.summary.scalar('learning_rate', lr)
    opt = tf.train.AdamOptimizer(lr)

    train_vars = list(set(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)) - set(vis_model.model_weights))
    train_op = opt.minimize(loss, var_list=train_vars, name='train_op') 

    global_saver = tf.train.Saver()
    train_writer = tf.summary.FileWriter(log_file, sess.graph)
    print('Model is built')

Initialising for first time
Building Text Model...
use elmo for textual model
Instructions for updating:
Colocations handled automatically by placer.


W0512 13:41:21.421391 139673951896960 deprecation.py:323] From /home/david_hc95/anaconda3/envs/grounding/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0512 13:41:22.081396 139673951896960 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


...

Instructions for updating:
Use keras.layers.dense instead.


W0512 13:41:22.450596 139673951896960 deprecation.py:323] From <ipython-input-7-147e699d0ac6>:32: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


Building Visual Model...
Instructions for updating:
Use keras.layers.conv2d instead.


W0512 13:41:22.805313 139673951896960 deprecation.py:323] From ../utilities/utils_model.py:197: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.conv2d instead.


use cosine simliarity attn mechanism
use pyramid
Model is built


In [8]:
for x in train_vars:
    print(x)

<tf.Variable 'stack_v/v2/kernel:0' shape=(3, 3, 512, 512) dtype=float32_ref>
<tf.Variable 'stack_v/v3/kernel:0' shape=(3, 3, 512, 512) dtype=float32_ref>
<tf.Variable 'text_image_fusion/dense_1/kernel:0' shape=(1024, 1024) dtype=float32_ref>
<tf.Variable 'dense_1/kernel:0' shape=(1024, 1024) dtype=float32_ref>
<tf.Variable 'text_image_fusion/dense_1/bias:0' shape=(1024,) dtype=float32_ref>
<tf.Variable 'dense_1/bias:0' shape=(1024,) dtype=float32_ref>
<tf.Variable 'stack_v/v4/kernel:0' shape=(3, 3, 512, 512) dtype=float32_ref>
<tf.Variable 'BiLSTM/bidirectional_rnn/bw/lstm_cell/bias:0' shape=(2048,) dtype=float32_ref>
<tf.Variable 'BiLSTM/bidirectional_rnn/bw/lstm_cell/kernel:0' shape=(1024, 2048) dtype=float32_ref>
<tf.Variable 'module/aggregation/weights:0' shape=(3,) dtype=float32>
<tf.Variable 'text_image_fusion/dense_2/bias:0' shape=(1024,) dtype=float32_ref>
<tf.Variable 'module/aggregation/scaling:0' shape=() dtype=float32>
<tf.Variable 'dense/bias:0' shape=(1024,) dtype=float32

# Batch Gen

In [9]:
def batch_gen_vqa(valid_ids, annot_dict, train_images,n_batch):
    img_batch = np.empty((n_batch, 299, 299, 3), dtype='float32')
    cap_batch = []
    ans_batch = []
    chosen_ids = random.sample(valid_ids, n_batch)
    for i, chosen_id in enumerate(chosen_ids):
        imgbin = train_images[chosen_id]
        img_batch[i,:,:,:] = imgbin
        questions = []
        answers = []
        for annot in annot_dict[chosen_id]:
            q_id = list(annot.keys())[0] 
            questions.append(annot[q_id]['question'])
            label = np.zeros(3133, dtype='float32')
            for answer in annot[q_id]['answer_id']:
                label[answer[0]] = answer[1]
            answers.append(label)
        n_ques = len(questions)
        idx = random.choice(range(n_ques))  
        cap_batch.append(questions[idx])
        ans_batch.append(answers[idx])
        
    return img_batch, cap_batch, ans_batch

# Batch Gen

In [11]:
valid_ids_train, train_images = preprocess_images_vqa(ids_train, txn, txn_test)    
print('Done pre-processing training images')

Processed:  0  images
Processed:  10000  images
Processed:  20000  images
Processed:  30000  images
Processed:  40000  images
Processed:  50000  images
Processed:  60000  images
Processed:  70000  images
Processed:  80000  images
Processed:  90000  images
Processed:  100000  images
Processed:  110000  images
Processed:  120000  images
Done pre-processing images
Done pre-processing training images


In [ ]:
# Preprocessing Training Images
# valid_ids_train, train_images = preprocess_images(ids_train[:100], txn, txn_test)
# print('Done pre-processing training images')

# Training

In [12]:
# print('Initializing...')
_ = sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
#loading pretrained inception weights
print('Loading visual path model...')
vis_model.load_weights()
print('Model Loaded')

Loading visual path model...
Instructions for updating:
Use standard file APIs to check for files with this prefix.


W0512 14:01:48.980753 139673951896960 deprecation.py:323] From /home/david_hc95/anaconda3/envs/grounding/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1266: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from /home/david_hc95/grounding/iccv19_grounding/models/vgg_16.ckpt


I0512 14:01:48.985939 139673951896960 saver.py:1270] Restoring parameters from /home/david_hc95/grounding/iccv19_grounding/models/vgg_16.ckpt


Model Loaded


In [13]:
#loop on training data
print('Start training...')
max_class_acc = 0
val_acc = np.zeros((n_epochs,))
train_loss = np.zeros((n_epochs,))
#bert_embedding = BertEmbedding()

for e in range(n_epochs):
    print('\n\n=====Epoch: %d'%e)
    avg_loss = 0
    
    if e < 9:
        lr_value = lr_value
    elif 9 <= e < 14:
        lr_value = lr_value / 2.0
    elif e >= 14:
        lr_value = lr_value/ 4.0
        
    print('===Train')
    for i in range(n_iter_per_epoch):   
        
        if text_model == 'elmo':
            img_batch, cap_batch, ans_batch = batch_gen_vqa(valid_ids_train, dict_train, train_images,n_batch)
            feed_dict = {input_img: img_batch, text_batch: cap_batch, answer_batch: ans_batch, mode: 'train', lr: lr_value}
        
        loss_val, _ = sess.run([loss, train_op], feed_dict)
        avg_loss += loss_val
        var = [i * n_batch, n_iter_per_epoch * n_batch, avg_loss / float(i + 1)]
        prnt = 'Sample {}/{}, train_loss:{:.4f} \r'.format(var[0], var[1], var[2])
        sys.stdout.write(prnt)
        sys.stdout.flush()
        train_loss[e] = avg_loss/float(n_iter_per_epoch+1)
        
    if e % 5 == 0:
        print('saving model')
        global_saver.save(sess, model_name + str(e))
        print('model saved')
          
print('Training done.')
#saving the session
print('Saving model...')
global_saver.save(sess,  model_name + str(n_epochs))
print('Saving done.')

Start training...


=====Epoch: 0
===Train
saving model2/123264, train_loss:0.0043 
model saved


=====Epoch: 1
===Train
Sample 123232/123264, train_loss:0.0020 

=====Epoch: 2
===Train
Sample 123232/123264, train_loss:0.0018 

=====Epoch: 3
===Train
Sample 123232/123264, train_loss:0.0016 

=====Epoch: 4
===Train


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



saving model2/123264, train_loss:0.0015 
model saved


=====Epoch: 6
===Train
Sample 123232/123264, train_loss:0.0014 

=====Epoch: 7
===Train
Sample 123232/123264, train_loss:0.0014 

=====Epoch: 8
===Train
Sample 123232/123264, train_loss:0.0014 

=====Epoch: 9
===Train
Sample 123232/123264, train_loss:0.0013 

=====Epoch: 10
===Train
saving model2/123264, train_loss:0.0012 
model saved


=====Epoch: 11
===Train
Sample 123232/123264, train_loss:0.0012 

=====Epoch: 12
===Train
Sample 123232/123264, train_loss:0.0012 

=====Epoch: 13
===Train
Sample 123232/123264, train_loss:0.0012 

=====Epoch: 14
===Train
Training done.123264, train_loss:0.0012 
Saving model...
Saving done.


In [ ]:
plt.figure()
plt.plot(train_loss, label='Train loss '+ model_name)
plt.legend()
plt.show()
plt.figure()
plt.plot(100. * val_acc, label='Validation classification_acc '+ model_name)
plt.legend()
plt.show()

# Loading Model for Testing

In [5]:
print('test')

test


In [8]:
try:
    sess.close()
except NameError:
    print("Definining session for the first time")
#tf.reset_default_graph()

Definining session for the first time


In [9]:
condition = 'Model_VQA_LSTM_LearnableFuse_VQA2_VGG15'
print('Loading grounding pretrained model...')
model_path = './my_model/'+ condition
print(model_path)
sess, graph = load_model(model_path,config)
print('Model Loaded')

Loading grounding pretrained model...
./my_model/Model_VQA_LSTM_LearnableFuse_VQA2_VGG15
Instructions for updating:
Use standard file APIs to check for files with this prefix.


W0513 00:17:49.362160 139913678259584 deprecation.py:323] From /home/david_hc95/anaconda3/envs/grounding/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1266: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from ./my_model/Model_VQA_LSTM_LearnableFuse_VQA2_VGG15


I0513 00:17:49.367622 139913678259584 saver.py:1270] Restoring parameters from ./my_model/Model_VQA_LSTM_LearnableFuse_VQA2_VGG15


Model Loaded


In [10]:
input_img = sess.graph.get_tensor_by_name("input_img:0")
text_batch = sess.graph.get_tensor_by_name("text_input:0")
mode = sess.graph.get_tensor_by_name("mode:0")
fusion = sess.graph.get_tensor_by_name("text_image_fusion/fusion:0")

# Testing

In [11]:
# Preprocessing Testing Images
valid_ids_test, test_images = preprocess_images_vqa(ids_test, txn_test, txn)
print('Done pre-processing test images')

Processed:  0  images
Processed:  10000  images
Processed:  20000  images
Processed:  30000  images
Processed:  40000  images
Processed:  50000  images
Processed:  60000  images
Processed:  70000  images
Processed:  80000  images
Done pre-processing images
Done pre-processing test images


In [19]:
class Question:
    def __init__(self,question_id,image_id,question,answer_id=None):
        self.question_id = question_id
        self.image_id = image_id
        self.question = question
        self.answer_id = answer_id
        
def generate_batches(questions_list, batch_size):
    n = len(questions_list)
    
    questions_batch = []
    for i, ques in enumerate(questions_list):
        questions_batch.append(ques)
        if ((i + 1) % batch_size == 0) or ((i + 1) == n):
            yield questions_batch
            questions_batch = []

def validate_test(final_test_list, test_images, idx2ans, batch_size):
    results = list()
    print("----started")

    l = 0
    for question_batch in generate_batches(final_test_list, batch_size):
        num_questions = len(question_batch)
        img_batch = np.empty((num_questions, 299, 299, 3))
        ques_batch = []
        ques_ids_batch = []
        
        for j, ques in enumerate(question_batch):
            image_id = ques.image_id
            question_id = ques.question_id
            question = ques.question

            img = np.reshape(test_images[image_id], (299, 299, 3))
            img_batch[j,:] = img
            ques_batch.append(question)
            ques_ids_batch.append(question_id)
        
        feed_dict = {input_img: img_batch, text_batch: ques_batch, mode:"test"}
        pred = sess.run(fusion, feed_dict)
        idx = np.argmax(pred, axis=1)
        for k, each_idx in enumerate(idx):
            results.append({"question_id": ques_ids_batch[k], "answer": idx2ans[each_idx]}) 
        l += 1
        if l % 1000 == 0:
            print("Number of batches done: ", l, "; num_results: ", str(len(results)))
    return results

In [ ]:
#Generating All Questions in Answer Batch
final_test_list = []
for k, doc_id in enumerate(dict_test):
    image_id = doc_id
    for annot in dict_test[doc_id]:
        question_id = list(annot.keys())[0]
        question = annot[question_id]['question']
        ques = Question(question_id ,image_id, question)
        final_test_list.append(ques)

results = validate_test(final_test_list, test_images, idx2ans, batch_size=64)

----started
Number of batches done:  100 ; num_results:  6400


In [ ]:
print(len(results))

In [ ]:
#Saving as json file to fetch to server
import json 
with open('model_'+ condition + '20_results.json', 'w') as fp:
    json.dump(results, fp)